In [10]:
import torch
import torch.nn as nn
import math
from einops import reduce
import numpy as np
# 示例 2: 从向量创建对角矩阵 (返回2D tensor)
vector = torch.tensor([0, 20, 30])
diagonal_matrix = torch.diag(vector)
print("\n原向量:\n", vector)
print("从向量创建的对角矩阵:\n", diagonal_matrix)
# 输出:
# tensor([[10,  0,  0],
#         [ 0, 20,  0],
#         [ 0,  0, 30]])


原向量:
 tensor([ 0, 20, 30])
从向量创建的对角矩阵:
 tensor([[ 0,  0,  0],
        [ 0, 20,  0],
        [ 0,  0, 30]])


In [29]:
import torch
import torch.nn as nn
import math
from einops import reduce
from einops import einsum
import numpy as np
class rope(nn.Module):
    def __init__(self, theta: float, d_k: int, max_seq_len: int, device=None):
        super().__init__()
        THETA = torch.tensor([math.pow(theta, (2*k)/d_k) for k in range(int(d_k/2))])

        R_list = []
        for j in range(max_seq_len):
            R = np.zeros((d_k, d_k), dtype=np.float32)
            R[0::2, 0::2] = np.diag(torch.cos(j/THETA))  
            R[0::2, 1::2] = np.diag(-torch.sin(j/THETA))   
            R[1::2, 0::2] = np.diag(torch.sin(j/THETA)) 
            R[1::2, 1::2] = np.diag(torch.cos(j/THETA))
            R_list.append(R)
        self.register_buffer("RoPE" ,torch.tensor(R_list, dtype=torch.float32),persistent=False)
    def forward(self, x: torch.Tensor, token_positions: torch.Tensor) -> torch.Tensor:
        return einsum(x, self.RoPE[token_positions], "... sequence_length d_k, ... sequence_length d_k dk -> ... sequence_length dk")

In [30]:
a= rope(3, 6, 8)
a.RoPE

tensor([[[ 1.0000, -0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  1.0000, -0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  1.0000, -0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000]],

        [[ 0.5403, -0.8415,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.8415,  0.5403,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.7691, -0.6391,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.6391,  0.7691,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.8866, -0.4624],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.4624,  0.8866]],

        [[-0.4161, -0.9093,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.9093, -0.4161,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.1830, -0.9831,  0.0000,  0.0000],
         [ 0.0000,  0